In [13]:
import pickle
import numpy as np

import glob
import pandas as pd

fl = sorted(glob.glob('../features/*.pkl'))
for l in fl:
    print("'{}',".format(l))

'../features/cnn_1_aug1_feat.pkl',
'../features/cnn_1_aug_1_fold8_feat.pkl',
'../features/cnn_1_aug_rescale_preprocess_feat.pkl',
'../features/cnn_1_aug_skimage_denoise_feat.pkl',
'../features/cnn_1_aug_skimage_preprocess_feat.pkl',
'../features/cnn_1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
'../features/cnn_2_aug_1_fold8_feat.pkl',
'../features/cnn_2_aug_denoise_preprocess_feat.pkl',
'../features/cnn_2_aug_rescale_preprocess_feat.pkl',
'../features/cnn_2_aug_skimage_preprocess_feat.pkl',
'../features/cnn_2_feat.pkl',
'../features/cnn_3_aug1_feat.pkl',
'../features/cnn_3_aug_1_fold8_feat.pkl',
'../features/cnn_3_aug_denoise_preprocess_feat.pkl',
'../features/cnn_3_aug_rescale_preprocess_feat.pkl',
'../features/cnn_3_aug_skimage_preprocess_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/cnn_4_aug1_feat_add_early.pkl',
'../features/cnn_4_aug_1_fold8_feat.pkl',
'../features/cnn_4_aug_denoise_preprocess_feat.pkl',
'../features/cnn_4_aug_rescale_preprocess_feat.pkl',
'../fea

In [14]:
def load_x_y():
    train_x,test_x=[],[]
    for f in fl:
        #print(f)
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 81) (8424, 81) (1604,)


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]

        # train model
        m = LogisticRegression()
        m.fit(X_train,y_train)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict_proba(X_test))
        curr_tr_loss = log_loss(y_train,m.predict_proba(X_train))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        print('curr train loss',curr_tr_loss,'curr val loss', curr_val_loss)
        
        if test_pred is None:
            test_pred = m.predict_proba(test_x)
            w_test_pred = m.predict_proba(test_x)*curr_r_val_loss
        else:
            curr_pred = m.predict_proba(test_x)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    print(test_pred.shape)
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame(columns=['id','is_iceberg'])
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred[:,1]
        print(submission.head())
        submission.to_csv('../results/all_lr_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission['is_iceberg']=w_test_pred[:,1]
        print(submission.head())
        submission.to_csv('../results/weighted_lr_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

In [16]:
cv_test(3,42,True)

curr train loss 0.121489485896 curr val loss 0.161220116863
curr train loss 0.120532264156 curr val loss 0.157156100942
curr train loss 0.118121620929 curr val loss 0.168833538171
local average valid loss 0.162403251992 val loss std 0.00484014404794
(8424, 2)
         id  is_iceberg
0  5941774d    0.011834
1  4023181e    0.800866
2  b20200e4    0.060130
3  e7f018bb    0.992343
4  4371c8c3    0.520726
         id  is_iceberg
0  5941774d    0.011785
1  4023181e    0.801162
2  b20200e4    0.060697
3  e7f018bb    0.992369
4  4371c8c3    0.521647


In [17]:
cv_test(5,42,True)

curr train loss 0.121353564752 curr val loss 0.168377142044
curr train loss 0.127173648931 curr val loss 0.138150706551
curr train loss 0.121630336465 curr val loss 0.166146758967
curr train loss 0.128969595744 curr val loss 0.138714470399
curr train loss 0.118095975335 curr val loss 0.180360238539
local average valid loss 0.1583498633 val loss std 0.016966726891
(8424, 2)
         id  is_iceberg
0  5941774d    0.011346
1  4023181e    0.788997
2  b20200e4    0.051450
3  e7f018bb    0.992099
4  4371c8c3    0.479738
         id  is_iceberg
0  5941774d    0.011381
1  4023181e    0.791712
2  b20200e4    0.054284
3  e7f018bb    0.992156
4  4371c8c3    0.489019


In [18]:
cv_test(3,252,True)

curr train loss 0.11504411745 curr val loss 0.176099936678
curr train loss 0.120782530831 curr val loss 0.159173131512
curr train loss 0.120636235176 curr val loss 0.161831994215
local average valid loss 0.165701687468 val loss std 0.00743236515447
(8424, 2)
         id  is_iceberg
0  5941774d    0.011121
1  4023181e    0.788731
2  b20200e4    0.163219
3  e7f018bb    0.992280
4  4371c8c3    0.523322
         id  is_iceberg
0  5941774d    0.011125
1  4023181e    0.794758
2  b20200e4    0.166800
3  e7f018bb    0.992213
4  4371c8c3    0.531448


In [19]:
cv_test(5,66,True)

curr train loss 0.123599804306 curr val loss 0.152036961168
curr train loss 0.119206879091 curr val loss 0.174399826345
curr train loss 0.12643009826 curr val loss 0.145396785003
curr train loss 0.121342071709 curr val loss 0.170138677713
curr train loss 0.127980228221 curr val loss 0.136111242107
local average valid loss 0.155616698467 val loss std 0.0145699536873
(8424, 2)
         id  is_iceberg
0  5941774d    0.013027
1  4023181e    0.787112
2  b20200e4    0.057496
3  e7f018bb    0.992034
4  4371c8c3    0.477261
         id  is_iceberg
0  5941774d    0.013175
1  4023181e    0.785762
2  b20200e4    0.057877
3  e7f018bb    0.991869
4  4371c8c3    0.476128


In [20]:
cv_test(10,42,True)

curr train loss 0.120319500886 curr val loss 0.195590372718
curr train loss 0.126939329683 curr val loss 0.147522889249
curr train loss 0.125974840789 curr val loss 0.150496182728
curr train loss 0.126893110067 curr val loss 0.130027336743
curr train loss 0.118943272443 curr val loss 0.21194658441
curr train loss 0.129243002347 curr val loss 0.116962705284
curr train loss 0.123624738088 curr val loss 0.181040668855
curr train loss 0.130371923206 curr val loss 0.105794068777
curr train loss 0.122850862868 curr val loss 0.182407767002
curr train loss 0.12243698971 curr val loss 0.176234398524
local average valid loss 0.159802297429 val loss std 0.0333286032661
(8424, 2)
         id  is_iceberg
0  5941774d    0.011658
1  4023181e    0.788168
2  b20200e4    0.049522
3  e7f018bb    0.991966
4  4371c8c3    0.477538
         id  is_iceberg
0  5941774d    0.011668
1  4023181e    0.785544
2  b20200e4    0.048550
3  e7f018bb    0.991918
4  4371c8c3    0.481422
